In [1]:
from qoop.core import state, ansatz, metric
import qiskit
from qiskit import transpile
from qiskit.quantum_info import Operator, DensityMatrix, Kraus
from scipy.linalg import qr
import numpy as np
import tensorflow as tf

In [2]:
# Step 1: Read quantum compilation and learn how to use qoop
# https://github.com/vutuanhai237/qoop/wiki/Advance:-Custom-state-preparation

# Step 2: Implement the following function

def V(num_qubits: int):
    '''
        Create a ansatz V with n qubits and assign its parameters
    '''
    #Assign random parameter
    circuit = ansatz.polygongraph (num_qubits=num_qubits)
    num_params = circuit.num_parameters
    x0 = 2 * np.pi * np.random.random(num_params)
    circuit = circuit.assign_parameters(dict(zip(circuit.parameters, x0)))
    return circuit

# def Epsilon(rho, kraus_operators):
#     # K = K_noise = [\sqrt(p) I @ I, \sqrt(1-p) Z @ Z]
#     # see Eq. 1 Ref. [1]
#     return sum(K @ rho.data @ np.transpose(np.conjugate(K)) for K in kraus_operators)

# def Epsilon2(rho, unitary_matrix):
#     # K = K_noise = [\sqrt(p) I @ I, \sqrt(1-p) Z @ Z]
#     # see Eq. 1 Ref. [1]
#     return (np.transpose(np.conjugate(unitary_matrix)) @ rho.data @ unitary_matrix)

def calRho3 (rho, unitary_matrix, kraus_operators):
    rho2 = sum(K @ rho @ np.transpose(np.conjugate(K)) for K in kraus_operators)
    rho3 = (np.transpose(np.conjugate(unitary_matrix)) @ rho2 @ unitary_matrix)
    return rho3

def createKraus(unitary_matrix):
    '''
        Create a set of Kraus Operators from the input unitary matrix, using QR decomposition
    '''
    kraus_ops = []
    Q, R = qr(unitary_matrix)

    #Q: a 2^N x 2^N matrix, N is the number of qubits
    for q in Q:
        q = np.expand_dims(q, 1)
        kraus_ops.append(q @ np.transpose(np.conjugate(q)))
    return tf.convert_to_tensor(kraus_ops)

def compilation_trace_fidelity(rho, sigma):
    """Calculating the fidelity metric

    Args:
        - rho (DensityMatrix): first density matrix
        - sigma (DensityMatrix): second density matrix

    Returns:
        - float: trace metric has value from 0 to 1
    """
    rho2 = tf.linalg.sqrtm((tf.linalg.sqrtm(rho)) @ (rho))

    # Cast to a supported type
    real_part = tf.math.real(rho2)
    imaginary_part = tf.math.imag(rho2)

    # Check for NaNs in both real and imaginary parts
    contains_nan_real = tf.reduce_any(tf.math.is_nan(real_part))
    contains_nan_imag = tf.reduce_any(tf.math.is_nan(imaginary_part))

    contains_nan = contains_nan_real or contains_nan_imag

    if contains_nan == True:
        rho2 = rho

    return tf.linalg.trace(
            rho2
            @ (tf.linalg.sqrtm(sigma))
        )

#cost func to compare 2 given rhos
def cost(rho, rho3):
    return 1-compilation_trace_fidelity(rho, rho3) 


In [3]:
n = 5

#Create a ansatz V with N qubits
circuit = V(n)

print(circuit)

     ┌─────────────┐   ┌────────────┐                     ░ 
q_0: ┤ Ry(0.82161) ├─■─┤ Ry(4.1108) ├──────────────────■──░─
     └┬────────────┤ │ ├───────────┬┘                  │  ░ 
q_1: ─┤ Ry(6.1722) ├─■─┤ Ry(3.072) ├──■────────────────┼──░─
      ├────────────┤   ├───────────┴┐ │                │  ░ 
q_2: ─┤ Ry(5.0437) ├─■─┤ Ry(1.8067) ├─■────────────────┼──░─
      ├────────────┤ │ ├────────────┤                  │  ░ 
q_3: ─┤ Ry(3.8493) ├─■─┤ Ry(6.2051) ├─■────────────────┼──░─
      ├────────────┤   └────────────┘ │ ┌────────────┐ │  ░ 
q_4: ─┤ Ry(5.6669) ├──────────────────■─┤ Ry(3.6808) ├─■──░─
      └────────────┘                    └────────────┘    ░ 
c: 5/═══════════════════════════════════════════════════════
                                                            


In [4]:
# Get the unitary operator corresponding to the circuit
unitary_op = Operator(circuit)

# Get the unitary matrix
unitary_matrix = unitary_op.data
print (unitary_matrix.shape) #2^N x 2^N

(32, 32)


In [5]:
# Initialize a set of Kraus Operators from the given circuit
KrausOperators = createKraus(unitary_matrix=unitary_matrix)

#print(sum(K @ np.transpose(np.conjugate(K)) for K in KrausOperators))


In [6]:
#Initialize rho
rho = DensityMatrix.from_label('0' * n)
rho = tf.convert_to_tensor(rho)

#ca
rho3 = calRho3(rho=rho, kraus_operators=KrausOperators, unitary_matrix=unitary_matrix) 

#print(rho)
#print(rho3)


In [7]:
print(cost(rho=rho, rho3=rho3))

tf.Tensor((0.9934031823903812+0j), shape=(), dtype=complex128)


In [8]:
# Step 3: Implement the following function
# state_need_tomogaphy = ...
# rho = np.conjugate(np.transpose(state_need_tomogaphy)) @ state_need_tomogaphy   # density matrix
# rho' = Delta(rho)
# compiler = qoop.qcompilation.QuantumCompilation(U = rho', V = V())
# compiler.fit()
# compiler.plot()
# see fidelities versus iteration

In [9]:
#Auto Diff
'''
    rho: The initial rho
    unitary_matrix: randomly generated circle in initialization step
    kraus_operators: set of kraus operators
    n: number of qubits
    alpha: learning rate (?)'''
def Derivative(rho, unitary_matrix, kraus_operators, n, alpha=0.1):
    tensorKraus = tf.Variable(kraus_operators)
    with tf.GradientTape() as tape:
        y = calRho3(rho, unitary_matrix, tensorKraus)
        f = 1 - compilation_trace_fidelity(rho, y) 
    
    # Get the gradient of y with respect to x
    c = tape.gradient(f, tensorKraus)

    # Reshape c to kN * N matrix
    c = tf.reshape(c, (2**n * 2**n,2**n))

    # Reshape kraus_operators to kN * N matrix
    kraus_operators = tf.reshape(kraus_operators, (2**n * 2**n,2**n))
    
    # Compute the projection term
    proj = c - kraus_operators @ (np.transpose(np.conjugate(c)) @ kraus_operators  + np.transpose(np.conjugate(kraus_operators)) @ c) / 2

    # Update the Kraus operators
    updated_kraus_operators = kraus_operators - alpha * proj
    
    # Return the updated Kraus operators
    return updated_kraus_operators



In [10]:
KrausOperatorsTry = tf.identity(KrausOperators)

In [11]:
# try looping manually
for i in range (0, 1000):
    _cost = cost(rho, calRho3(rho, unitary_matrix, KrausOperatorsTry))

    #Update Kraus Operators
    KrausOperatorsTry=Derivative(rho, unitary_matrix, KrausOperatorsTry, n)

    #Reshape
    KrausOperatorsTry = tf.reshape(KrausOperatorsTry, (2**n,2**n,2**n))

    #print('K(t)K\n',sum(K @ np.transpose(np.conjugate(K)) for K in KrausOperatorsTry)) # I

    print (_cost)

tf.Tensor((0.9934031823903812+0j), shape=(), dtype=complex128)
tf.Tensor((0.936980982797803+0j), shape=(), dtype=complex128)
tf.Tensor((0.8870161941445052+0j), shape=(), dtype=complex128)
tf.Tensor((0.8390045490252908+0j), shape=(), dtype=complex128)
tf.Tensor((0.7918233397525076+0j), shape=(), dtype=complex128)
tf.Tensor((0.7455524318489803+0j), shape=(), dtype=complex128)
tf.Tensor((0.7003558836120231+0j), shape=(), dtype=complex128)
tf.Tensor((0.6564008338568985+0j), shape=(), dtype=complex128)
tf.Tensor((0.6138404945742832+0j), shape=(), dtype=complex128)
tf.Tensor((0.572808460986199+0j), shape=(), dtype=complex128)
tf.Tensor((0.5334161797953821+0j), shape=(), dtype=complex128)
tf.Tensor((0.49575188767414036+0j), shape=(), dtype=complex128)
tf.Tensor((0.4598800171366244+0j), shape=(), dtype=complex128)
tf.Tensor((0.42584242896780633+0j), shape=(), dtype=complex128)
tf.Tensor((0.39366126965519455+0j), shape=(), dtype=complex128)
tf.Tensor((0.36333865698703816+0j), shape=(), dtype=co

In [12]:
print(sum(K @ np.transpose(np.conjugate(K)) for K in KrausOperatorsTry))
print(KrausOperatorsTry)

tf.Tensor(
[[ 7.68946905e-01+0.00000000e+00j -4.79620517e-02-5.76840326e-23j
  -1.69235945e-02+1.11318497e-21j ...  6.45419467e-04-4.69881324e-21j
   1.75094144e-03+5.21457321e-20j  1.25670852e-03+4.04968588e-20j]
 [-4.79620517e-02+5.76840326e-23j  7.88967823e-01+0.00000000e+00j
   1.55818228e-03-1.30739371e-21j ...  3.49894062e-03+2.79231229e-21j
  -8.59353304e-04-3.96598964e-20j -7.71296774e-04-2.88308985e-20j]
 [-1.69235945e-02-1.11318497e-21j  1.55818228e-03+1.30739371e-21j
   9.95591210e-01+0.00000000e+00j ...  1.12360963e-03+5.26116103e-20j
  -1.18682346e-02-4.16515407e-19j -9.36423608e-03-3.61474241e-19j]
 ...
 [ 6.45419467e-04+4.69881324e-21j  3.49894062e-03-2.79231229e-21j
   1.12360963e-03-5.26116103e-20j ...  1.00338219e+00+0.00000000e+00j
  -3.82369949e-02-7.06390351e-19j -3.12257481e-02-3.88170157e-19j]
 [ 1.75094144e-03-5.21457321e-20j -8.59353304e-04+3.96598964e-20j
  -1.18682346e-02+4.16515407e-19j ... -3.82369949e-02+7.06390351e-19j
   1.37728777e+00+0.00000000e+00j  3

In [13]:
#Check the output
rho_out_1 = sum(K @ rho @ np.transpose(np.conjugate(K)) for K in KrausOperatorsTry)
rho_out_2 = (unitary_matrix @ rho @ np.transpose(np.conjugate(unitary_matrix)))

In [14]:
print(compilation_trace_fidelity(rho_out_1, rho_out_2))

tf.Tensor((0.9999999999999997+3.887232962256389e-19j), shape=(), dtype=complex128)
